In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

In [ ]:
!pip install matplotlib-venn
!pip install --upgrade langchain transformers
!pip install --upgrade langchain
!pip install langchain[huggingface]
!pip install langchain_community
!pip install transformers

In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap

In [ ]:
# ----------------- Imports -----------------
import torch
import textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

# ----------------- Prompt Constants -----------------
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"

AYURVEDA_SYSTEM_PROMPT = """\
You are an advanced Ayurveda expert and therapist that excels at giving advice.
Always answer as helpfully as possible. Your answers should be natural, clear, safe, and grounded in Ayurvedic principles.
Avoid emojis. Do not provide medical diagnoses, but do recommend Ayurvedic remedies, therapies, herbs, and wellness routines.
If you are unsure or the question is unclear, politely say you cannot answer."""

# ----------------- Prompt Generator -----------------
def get_prompt(instruction, new_system_prompt=AYURVEDA_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
    if citation:
        prompt_template += f"\n\nCitation: {citation}"
    return prompt_template

def remove_substring(string, substring):
    return string.replace(substring, "")

# ----------------- Model Loader -----------------
print("🔄 Loading model and tokenizer...")

tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    device_map='auto',
    torch_dtype=torch.float16,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("✅ Model loaded successfully.")

# ----------------- Generation Function -----------------
def generate(text, citation=None):
    instruction = (
        "Convert the following input text from a confused human to a well-reasoned and step-by-step Ayurvedic advice:\n\n"
        + text.strip()
    )
    prompt = get_prompt(instruction, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            temperature=0.7,
        )

    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    answer = remove_substring(output_text, prompt).strip()

    # Simple formatting: add markdown titles and bullets where relevant
    answer = post_process_answer(answer)
    return answer

# ----------------- Formatting Enhancer -----------------
def post_process_answer(text):
    # Add bold intro
    formatted = "**🪔 Ayurvedic Advice:**\n\n"

    # Bullet points formatting (basic heuristic)
    lines = text.split('\n')
    for line in lines:
        if any(keyword in line.lower() for keyword in ['herb', 'therapy', 'recommend', 'routine', 'diet']):
            formatted += f"- {line.strip()}\n"
        else:
            formatted += f"{line.strip()}\n"

    return formatted.strip()

# # ----------------- Gradio Interface -----------------

# custom_css = """
# .gradio-container {
#     max-width: 800px !important;
#     width: 90vw;
#     margin: 2rem auto;
#     border-radius: 8px;
#     overflow-y: auto;
#     border: 1px solid #ccc;
#     box-shadow: 0 0 10px rgba(0,0,0,0.05);
#     display: flex;
#     flex-direction: column;
#     height: 90vh;
# }
# #header {
#     background: #2e7d32;
#     color: white;
#     padding: 1rem;
#     text-align: center;
#     font-size: 1.5rem;
#     font-weight: bold;
#     border-radius: 8px 8px 0 0;
#     user-select: none;
# }
# #chat-window {
#     flex: 1;
#     overflow-y: auto;
#     padding: 1rem;
#     background: #e8f5e9;
#     display: flex;
#     flex-direction: column;
#     gap: 10px;
#     box-sizing: border-box;
# }
# .gr-chat-message {
#     max-width: 100% !important;
#     white-space: normal !important;
#     word-break: break-word !important;
#     display: block !important;
# }
# .gr-chat-message .markdown {
#     padding: 12px 18px;
#     border-radius: 20px;
#     line-height: 1.4;
#     width: 100%;
#     box-sizing: border-box;
# }
# .user-msg .markdown {
#     background-color: #2e7d32 !important;
#     color: white !important;
#     border-bottom-right-radius: 4px !important;
#     align-self: flex-end !important;
# }
# .bot-msg .markdown {
#     background-color: #c8e6c9 !important;
#     color: #2e7d32 !important;
#     border-bottom-left-radius: 4px !important;
#     align-self: flex-start !important;
# }
# #input-row {
#     border-top: 1px solid #ccc;
#     padding: 10px;
#     background: white;
#     display: flex;
#     align-items: center;
#     gap: 10px;
#     box-sizing: border-box;
# }
# #user-input {
#     font-size: 1rem;
#     padding: 12px 16px;
#     border: 1px solid #ccc;
#     border-radius: 25px;
#     resize: none;
#     outline: none;
#     font-family: Arial, sans-serif;
#     width: 100%;
#     min-height: 48px;
#     overflow-y: auto;
#     box-sizing: border-box;
# }
# #user-input:focus {
#     border-color: #2e7d32;
# }
# #send-btn {
#     height: 50px;
#     padding: 0 25px;
#     border-radius: 25px;
#     background-color: #2e7d32;
#     color: white;
#     font-weight: bold;
#     font-size: 1rem;
#     border: none;
#     cursor: pointer;
#     transition: background-color 0.3s;
#     white-space: nowrap;
# }
# #send-btn:hover {
#     background-color: #1b5e20;
# }
# """

# def respond(message, chat_history):
#     chat_history = chat_history or []
#     try:
#         bot_reply = generate(message)
#     except Exception as e:
#         bot_reply = f"**Error:** {str(e)}"
#     chat_history.append((message.strip(), bot_reply.strip()))
#     return chat_history, ""

# with gr.Blocks(css=custom_css, title="Ayurvedic Chatbot", theme=gr.themes.Soft()) as demo:
#     gr.HTML('<div id="header">🧘‍♀️ Ayurvedic Remedy Chatbot 🧘‍♂️</div>')

#     chat = gr.Chatbot(elem_id="chat-window", bubble_full_width=True)

#     with gr.Row(elem_id="input-row"):
#         user_input = gr.Textbox(
#             placeholder="Enter symptoms or disease names...",
#             lines=2,
#             elem_id="user-input",
#             show_label=False,
#             container=False,
#             scale=5
#         )
#         send_button = gr.Button("Send", elem_id="send-btn", scale=1)

#     send_button.click(respond, inputs=[user_input, chat], outputs=[chat, user_input])
#     user_input.submit(respond, inputs=[user_input, chat], outputs=[chat, user_input])

# demo.launch(share=True)


custom_css = """
.gradio-container {
    max-width: 800px !important;
    width: 90vw;
    margin: 2rem auto;
    border-radius: 8px;
    overflow-y: auto;
    border: 1px solid #ccc;
    box-shadow: 0 0 10px rgba(0,0,0,0.05);
    display: flex;
    flex-direction: column;
    height: 90vh;
}
#header {
    background: #2e7d32;
    color: white;
    padding: 1rem;
    text-align: center;
    font-size: 1.5rem;
    font-weight: bold;
    border-radius: 8px 8px 0 0;
    user-select: none;
}
#chat-window {
    flex: 1;
    overflow-y: auto;
    padding: 1rem;
    background: #e8f5e9;
    display: flex;
    flex-direction: column;
    gap: 10px;
    box-sizing: border-box;
}
.gr-chat-message {
    max-width: 100% !important;
    white-space: normal !important;
    word-break: break-word !important;
    display: block !important;
}
.gr-chat-message .markdown {
    padding: 12px 18px;
    border-radius: 20px;
    line-height: 1.4;
    width: 100%;
    box-sizing: border-box;
}
.user-msg .markdown {
    background-color: #2e7d32 !important;
    color: white !important;
    border-bottom-right-radius: 4px !important;
    align-self: flex-end !important;
}
.bot-msg .markdown {
    background-color: #c8e6c9 !important;
    color: #2e7d32 !important;
    border-bottom-left-radius: 4px !important;
    align-self: flex-start !important;
}
#input-row {
    border-top: 1px solid #ccc;
    padding: 10px;
    background: white;
    display: flex;
    align-items: center;
    gap: 10px;
    box-sizing: border-box;
}
#user-input {
    font-size: 1rem;
    padding: 12px 16px;
    border: 1px solid #ccc;
    border-radius: 25px;
    resize: none;
    outline: none;
    font-family: Arial, sans-serif;
    width: 100%;
    min-height: 48px;
    overflow-y: auto;
    box-sizing: border-box;
}
#user-input:focus {
    border-color: #2e7d32;
}
#send-btn {
    height: 50px;
    padding: 0 25px;
    border-radius: 25px;
    background-color: #2e7d32;
    color: white;
    font-weight: bold;
    font-size: 1rem;
    border: none;
    cursor: pointer;
    transition: background-color 0.3s;
    white-space: nowrap;
}
#send-btn:hover {
    background-color: #1b5e20;
}
footer {
    display: none !important;
}
"""


def respond(message, chat_history):
    chat_history = chat_history or []
    try:
        bot_reply = generate(message)
    except Exception as e:
        bot_reply = f"**Error:** {str(e)}"
    chat_history.append((message.strip(), bot_reply.strip()))
    return chat_history, ""

with gr.Blocks(css=custom_css, title="Ayurvedic Chatbot", theme=gr.themes.Soft()) as demo:
    gr.HTML('<div id="header">🧘‍♀️ Ayurvedic Remedy Chatbot 🧘‍♂️</div>')

    chat = gr.Chatbot(elem_id="chat-window", bubble_full_width=True)

    with gr.Row(elem_id="input-row"):
        user_input = gr.Textbox(
            placeholder="Enter symptoms or disease names...",
            lines=2,
            elem_id="user-input",
            show_label=False,
            container=False,
            scale=5
        )
        send_button = gr.Button("Send", elem_id="send-btn", scale=1)

    send_button.click(respond, inputs=[user_input, chat], outputs=[chat, user_input])
    user_input.submit(respond, inputs=[user_input, chat], outputs=[chat, user_input])

demo.launch(share=True)




🔄 Loading model and tokenizer...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded successfully.


/tmp/ipython-input-677193205.py:337: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(elem_id="chat-window", bubble_full_width=True)
/tmp/ipython-input-677193205.py:337: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chat = gr.Chatbot(elem_id="chat-window", bubble_full_width=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0fffdfbfb39c8f5e96.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
